In [1]:
import os
import re
import json
import requests
import warcio
import sqlite3
import random
import whatsincc.file_operations as cc_fo

In [6]:
con = sqlite3.connect("../data/cc_sample.db")
cur = con.cursor()
cur.execute("CREATE TABLE wet_records(warc_block_digest, target_url, warc_date, content_language, content_length, wet_path)")

In [7]:
with open("../data/available_crawls.json", "r") as f:
    crawls = json.load(f)

# For testing purpose:
crawls = [crawls[0]]

In [8]:
for crawl in crawls:
    wet_paths_url = f"https://data.commoncrawl.org/crawl-data/{crawl}/wet.paths.gz"
    wet_paths_file = cc_fo.download_file(wet_paths_url, decompress=True)

    sample_size = 5
    with open(wet_paths_file, "r") as f:
        wet_paths_sample = [line.rstrip() for line in random.sample(list(f), sample_size)]

    for wet_path in wet_paths_sample:
        wet_url = f"https://data.commoncrawl.org/{wet_path}"

        with requests.get(wet_url, stream=True) as r:
            for record in warcio.ArchiveIterator(r.raw):
                if record.rec_type == "conversion":
                    rec_block_digest = record.rec_headers.get_header('WARC-Block-Digest')
                    rec_date = record.rec_headers.get_header('WARC-Date')
                    rec_lan =  record.rec_headers.get_header('WARC-Identified-Content-Language')
                    rec_url = record.rec_headers.get_header('WARC-Target-URI')
                    rec_len = int(record.rec_headers.get_header('Content-Length'))
                    rec_content = record.content_stream().read().decode('utf-8')

                    data = (rec_block_digest, rec_url, rec_date, rec_lan, rec_len, wet_path)

                    cur.execute(f"INSERT INTO wet_records VALUES(?,?,?,?,?,?)", data)
            con.commit()